In [248]:
import xml.etree.cElementTree as ET
import os
import nltk
import string
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame
import nltk.data
from nltk import sent_tokenize

### Get Training Files for Parsing

In [249]:
# set to the appropriate folder on your local drive
wd = os.path.dirname(os.path.abspath('__file__'))
folders = ["\\Dataset\\training-RiskFactors-Complete-Set1\\", "\\Dataset\\training-RiskFactors-Complete-Set2\\"]

#datafolder.append(codefolder.replace("Code", "Dataset\\training-RiskFactors-Complete-Set1"))
#datafolder.append(codefolder.replace("Code", "Dataset\\training-RiskFactors-Complete-Set2"))

#print (datafolder)

filenames = []

for folder in folders:
    for file in os.listdir(str(wd)+folder):
        filename = os.fsdecode(os.fsencode((str(wd)+folder+file)))
        if filename.endswith( ('.xml') ): # select xml files
            #print(filename)
            filenames.append(filename)


In [250]:
print("There are in total, {} files as part of the training set. ".format(len(filenames)))

There are in total, 790 files as part of the training set. 


### Get Testing specific files

In [251]:
# set to the appropriate folder on your local drive
wd = os.path.dirname(os.path.abspath('__file__'))
datafolder = ["\\Dataset\\testing-RiskFactors-Complete\\"]
#print (datafolder)

testfilenames = []

for folder in datafolder:
    for file in os.listdir(str(wd)+folder):
        filename = os.fsdecode(os.fsencode((str(wd)+folder+file)))
        if filename.endswith( ('.xml') ): # select xml files
            #print(filename)
            testfilenames.append(filename)
            
print("There are {} test XML files for validating the model.".format(len(testfilenames)))


There are 514 test XML files for validating the model.


### Define function to get sentences and labels

In [252]:
def get_sentences(file):

    tree = ET.ElementTree(file=file)
    root = tree.getroot()

    text = root.find('TEXT').text
    sentences = [sent.split('\n') for sent in sent_tokenize(text) if sent!='\n']
    all_sentences = []

    for item in sentences:
        for sub_item in item:
            if sub_item.replace(' ','') != '':
                all_sentences.append(sub_item)    
    
    return all_sentences

### Function definition for processing a file

This takes in the filename, tag and attribute as inputs and generates sentences and the corresponding label to form the dataset for training the model.  The function 'get_sentences' is incorporated into the function below.

In [253]:
def process_file(file, tag, attribute):
    
    # get all sentences in the file
    tree = ET.ElementTree(file=file)
    root = tree.getroot()

    text = root.find('TEXT').text
    sentences = [sent.split('\n') for sent in sent_tokenize(text) if sent!='\n']
    all_sentences = []

    for item in sentences:
        for sub_item in item:
            if sub_item.replace(' ','') != '':
                all_sentences.append(sub_item)    
                
    #all_sent = get_sentences(file)
    sent_label = {}

    sub_tags = []
    for item in root.find("TAGS"):
        if item.tag  == tag:
            label = (item.tag + "." + item.attrib[attribute]).lower().replace(" ", "_")
        else:
            label = ""

        for sub_item in item.findall(item.tag):
            if (item.tag==tag) and ('text' in sub_item.attrib.keys()):
                sub_tags.append((sub_item.attrib['text'], sub_item.attrib[attribute]))


    count=0
    for sent in all_sentences:
        label='Other'
        for tag in set(sub_tags):
            if tag[0] in sent:
                label = tag[1]
                count += 1

        sent_label[sent] = label
        
    # return empty dict if no tag found in file
    # else, return the sentences with the labels
    if count==0:
        return {}
    else:
        return sent_label


In [254]:
def get_TrainingData(tag, attrib, filenames, pct_split=0.9):

    """
    All files in the list (which holds the list of files in the directory) are parsed
    through to generate the training / dev datasets for the tag/attribute in context.
    
    The tags and attributes are passed on to the function as parameters.
    
    Input: 
    filenames: names of the file to be read in for processing in a list object
    tag: tag, as identified in the annotation.  Ex: DIABETES, HYPERTENSION etc. (string)
    attribute: specific attribute within the tag, from which to extract the value from (string)
    
    Returns: 
    Dataframe of the train / dev datasets (for the tag/attribute)
    """
    
    # using a 90/10 split by default unless specified as parameter
    split_index = int(len(filenames)*pct_split)
    random.seed(42)
    random.shuffle(filenames)
    
    train_files=[]
    dev_files=[]
    train_sent = []
    train_labels = []
    dev_sent = []
    dev_labels = []

    for file in filenames[:split_index]:
        file_data = process_file(file=file, tag=tag, attribute=attrib)
        for i in range(0, len(file_data)):
            train_files.append(file)
            
        for key, value in file_data.items():
            train_sent.append(key)
            train_labels.append(value)
        
    for file in filenames[split_index:]:
        file_data = process_file(file=file, tag=tag, attribute=attrib)
        for i in range(0, len(file_data)):
            dev_files.append(file)
        
        for key, value in file_data.items():
            dev_sent.append(key)
            dev_labels.append(value)

    train_df = pd.DataFrame({'filename': train_files, 'sentence': train_sent, 'label': train_labels})
    dev_df = pd.DataFrame({'filename': dev_files, 'sentence': dev_sent, 'label': dev_labels})
    return train_df, dev_df


In [255]:
def get_TestData(tag, attrib, filenames):

    """
    All files in the list (which holds the list of files in the directory) are parsed
    through to generate the test datasets for the tag/attribute in context.
    
    The tags and attributes are passed on to the function as parameters.
    
    Input: 
    filenames: names of the file to be read in for processing in a list object
    tag: tag, as identified in the annotation.  Ex: DIABETES, HYPERTENSION etc. (string)
    attribute: specific attribute within the tag, from which to extract the value from (string)
    
    Returns: 
    Dataframe of the test dataset (for the tag/attribute)
    """
    
    test_files=[]
    test_sent = []
    test_labels = []

    for file in filenames:
        file_data = process_file(file=file, tag=tag, attribute=attrib)
        for i in range(0, len(file_data)):
            test_files.append(file)
            
        for key, value in file_data.items():
            test_sent.append(key)
            test_labels.append(value)
        
    test_df = pd.DataFrame({'filename': test_files, 'sentence': test_sent, 'label': test_labels})
    return test_df


In [256]:
def get_filename(fullpath):
    #print(fullpath)
    fullpath=fullpath.replace("C:\\Users\\sudha\\Documents\\W266-NLP\\Final-Project-W266\\Code\\Dataset\\training-RiskFactors-Complete-Set1\\", "")
    return fullpath.replace("C:\\Users\\sudha\\Documents\\W266-NLP\\Final-Project-W266\\Code\\Dataset\\training-RiskFactors-Complete-Set2\\", "")


In [257]:
def get_testfilename(fullpath):
    return fullpath.replace("C:\\Users\\sudha\\Documents\\W266-NLP\\Final-Project-W266\\Code\\Dataset\\testing-RiskFactors-Complete\\", "")


### Generate Data for Models

The goal is to generate the tokens and the corresponding labels specific to model of interest.  Below, we will be building the data for 'indicator' attribute for the following tags:

* DIABETES
* CAD
* HYPERTENSION
* HYPERLIPIDEMIA
* OBESE
* FAMILY_HIST

Update the tag / indicator to get the training / dev / test datasets for them

In [390]:
# get data for model #2
tag = 'DIABETES'
attribute = 'time'

# training & dev datasets
df_train, df_dev = get_TrainingData(tag, attribute, filenames)
df_train['file'] = df_train['filename'].apply(get_filename)
df_train.drop('filename', 1, inplace=True)

df_dev['file'] = df_dev['filename'].apply(get_filename)
df_dev.drop('filename', 1, inplace=True)

# test dataset
df_test = get_TestData(tag, attribute, testfilenames)
df_test['file'] = df_test['filename'].apply(get_testfilename)
df_test.drop('filename', 1, inplace=True)


In [391]:
df_test.head(3)

,sentence,label,file
0,Record date: 2080-02-18,Other,110-03.xml
1,SDU JAR Admission Note,Other,110-03.xml
2,Name: \t Yosef Villegas,Other,110-03.xml


In [392]:
df_train.head(3)

,sentence,label,file
0,Record date: 2081-10-28,Other,157-04.xml
1,"55 yo s/p admit for chest pain, PTCA and stent...",Other,157-04.xml
2,no enzyme bump,Other,157-04.xml


In [393]:
df_dev.head(3)

,sentence,label,file
0,Record date: 2061-10-03,Other,304-02.xml
1,BMH Cancer Center,Other,304-02.xml
2,Hematology Note,Other,304-02.xml


In [394]:
# check size of each of the datasets generated
print(df_train.shape)
print(df_dev.shape)
print(df_test.shape)

(47377, 3)
(4993, 3)
(35556, 3)


### Augment the data so that the minority class has good representation

One of the challenges seen is due to the fact that some of the classes, such as 'glucose' in Diabetes-Indicator classification is under-represented.  In an attempt to boost the representation of the minority class, we duplicate those examples in the training set and then do a shuffle of the entire training set so that they are well spread out in the training data.


In [395]:
def augment_data(df, mincount=600):
    
    labels = df_train['label'].unique()
    print("labels: ", labels)
    print("\n=============================")
    print("Value Counts before augmentation: \n", df['label'].value_counts())
    print("\n=============================")
    
    for label in labels:
        label_count = (df[df['label']==label]).shape[0]
        while label_count < 600:
            df = pd.concat([df, df[df['label']==label]], axis=0)
            label_count = (df[df['label']==label]).shape[0]

    # shuffle rows in the augmented dataset
    df = df.sample(frac=1).reset_index(drop=True)
    print("Value Counts after augmentation: \n", df['label'].value_counts())
    
    return df

In [396]:
df_train = augment_data(df_train)

labels:  ['Other' 'before DCT' 'during DCT' 'after DCT']

Value Counts before augmentation: 
 Other         46098
during DCT      680
after DCT       301
before DCT      298
Name: label, dtype: int64

Value Counts after augmentation: 
 Other         46098
before DCT     1192
during DCT      680
after DCT       602
Name: label, dtype: int64


### Write the TRAINING & DEV Datasets

Capture the TRAINING and DEV datasets for the tag/attribute in context and write to the appropriate folder for processing.


In [397]:
le = LabelEncoder()

# refer (https://blog.insightdatascience.com/using-bert-for-state-of-the-art-pre-training-for-natural-language-processing-1d87142c29e7)

# get the training set for BERT in the required format
df_train_bert = pd.DataFrame({'user_id':df_train.index,
            'label':le.fit_transform(df_train['label']),
            'alpha':['a']*df_train.shape[0],
            'text':df_train['sentence'].replace(r'\n',' ',regex=True)})


# get the dev set for BERT in the required format
df_dev_bert = pd.DataFrame({'user_id':df_dev.index,
            'label':le.fit_transform(df_dev['label']),
            'alpha':['a']*df_dev.shape[0],
            'text':df_dev['sentence'].replace(r'\n',' ',regex=True)})


 # Creating test dataframe according to BERT (CoLA Format)
df_test_bert = pd.DataFrame({'id':df_test.index,
                 'sentence':df_test['sentence'].replace(r'\n',' ',regex=True)})


In [398]:
# get train/dev sets for BERT-NER

# get the training set for BERT-NER in the required format (only token and label)
#df_train_bert_ner = DI_train[['token', 'label']]
#df_dev_bert_ner = DI_dev[['token', 'label']]


In [399]:
df_train_bert.head()

,user_id,label,alpha,text
0,0,0,a,Other review of systems-he has
1,1,0,a,"No HSM, No CVA tenderness."
2,2,0,a,MRN:\t236-07-19
3,3,0,a,pending
4,4,3,a,diabetes mellitus.


In [400]:
# set train and dev sets to be a sequential set of values
print("Unique labels in the training dataset: ", df_train_bert['label'].unique())
print("Value counts by labels (training dataset): \n", df_train_bert['label'].value_counts())
print("\n")
print("Unique labels in the dev dataset: ", df_dev_bert['label'].unique())
print("Value counts by labels (dev dataset): \n", df_dev_bert['label'].value_counts())


Unique labels in the training dataset:  [0 3 1 2]
Value counts by labels (training dataset): 
 0    46098
2     1192
3      680
1      602
Name: label, dtype: int64


Unique labels in the dev dataset:  [0 3 2 1]
Value counts by labels (dev dataset): 
 0    4858
3      83
2      30
1      22
Name: label, dtype: int64


In [401]:
df1 = pd.DataFrame(df_train_bert['label'].value_counts(), index=None, columns=['label'])
df2 = pd.DataFrame(df_train['label'].value_counts(), index=None, columns=['label'])
df1['Bert Label'] = df1.index
df1['File Label'] = df2.index

df1.sort_values('Bert Label', inplace=True)
df1

,label,Bert Label,File Label
0,46098,0,Other
1,602,1,after DCT
2,1192,2,before DCT
3,680,3,during DCT


In [402]:
test_write_path = 'data_for_bert_augmented/test_files_with_labels/'+tag.lower()+'_'+attribute[:3]
# save the test file with labels for the purpose of running confusion matrix
df_test.to_csv(test_write_path+'_testfile.csv')

# save the label mappings between original test file and how it corresponds in BERT
df1[['Bert Label', 'File Label']].to_csv(test_write_path+'_labelmapping.csv')

### Save the TRAINING & DEV data to folder

Write the files into appropriate folders, so that it can be uploaded to cloud for processing / building model 

In [403]:
write_path = 'data_for_bert_augmented/'+tag.lower()+'_'+attribute[:3]

# train / dev sets for TAG-ATTRIBUTE
df_train_bert.to_csv(write_path+'/train.tsv', sep='\t', index=False, header=False)
df_dev_bert.to_csv(write_path+'/dev.tsv', sep='\t', index=False, header=False)

# test set for TAG-ATTRIBUTE
df_test_bert.to_csv(write_path+'/test.tsv', sep='\t', index=False, header=True)


### Check folder & Get Training

The format of the datasets have been adjused so as to perform multi-class classification and predicted against the test dataset.  The dev set is used for calculating the accuracy.

In [404]:
df_test['label'].value_counts()

Other         34642
during DCT      496
before DCT      214
after DCT       204
Name: label, dtype: int64

In [405]:
df_test[1270:1280]

,sentence,label,file
1270,"Diabetes (insulin-dependent, 3/66 HBA1C was 8.90)",before DCT,117-02.xml
1271,2.,Other,117-02.xml
1272,Hypercholesterolemia,Other,117-02.xml
1273,3.,Other,117-02.xml
1274,Hypertension,Other,117-02.xml
1275,4.,Other,117-02.xml
1276,History of angina,Other,117-02.xml
1277,5.,Other,117-02.xml
1278,"History of cyst near scapular, upper left back",Other,117-02.xml
1279,6.,Other,117-02.xml
